Forms:
    10-K
    10-Q
    8-K
    Proxy Statement
    Forms 3, 4, 5
    Schedulem13D
    144

10-K:
    NLP:
        Business?
        Risk Factors
        Unresolved Staff Comments
        Properties
        Legal Proceedings
        Mine Safety Disclosures
        

In [21]:
import pandas as pd
import re
import requests
import unicodedata
from bs4 import BeautifulSoup



## Get txt document from SEC website

In [2]:
# gets the text doc with all the filings
htmlText = r"https://www.sec.gov/Archives/edgar/data/1318605/000119312511054847/0001193125-11-054847.txt"



#get response
response = requests.get(htmlText)

# parse response
soup = BeautifulSoup(response.content, 'lxml')

## Decode Text

In [ ]:
from unicodedata import normalize

print('%r' % normalize('NFD', u'\u00C7'))  # decompose: convert Ç to "C + ̧"
print('%r' % normalize('NFC', u'C\u0327')) # compose: convert "C + ̧" to Ç

In [22]:
def restore_windows_1252_characters(restore_string):
    """
        Replace C1 control characters in the Unicode string s by the
        characters at the corresponding code points in Windows-1252,
        where possible.
    """

    def to_windows_1252(match):
        try:
            return bytes([ord(match.group(0))]).decode('windows-1252')
        except UnicodeDecodeError:
            # No character at the corresponding code point: remove it.
            return ''
        
    return re.sub(r'[\u0080-\u0099]', to_windows_1252, restore_string)




# def remove_newlines(string):
#     # \p, \xa0

In [ ]:
soup

## Split up into Parts 1 - 4

PART I

Business 
Risk Factors
Unresolved Staff Comments
Properties
Legal Proceedings
Reserved  

PART II

Market for Registrant’s Common Equity, Related Stockholder Matters and Issuer Purchases of Equity Securities 
Selected Financial Data
Management’s Discussion and Analysis of Financial Condition and Results of Operations
Quantitative and Qualitative Disclosures About Market Risk
Financial Statements and Supplementary Data
Changes in and Disagreements with Accountants on Accounting and Financial Disclosure
Controls and Procedures 
Other Information

PART III

Directors, Executive Officers and Corporate Governance
Executive Compensation
Security Ownership of Certain Beneficial Owners and Management and Related Stockholder Matters
Certain Relationships and Related Transactions, and Director Independence
Principal Accountant Fees and Services

Exhibits and Financial Statement Schedules

In [3]:
master_document_dict = {}

# find all the documents in the filing.
for filing_document in soup.find_all('document'):
    # define the document type, found under the <type> tag, this will serve as our key for the dictionary.
    document_id = filing_document.type.find(text=True, recursive=False).strip()
    # here are the other parts if you want them.
    document_sequence = filing_document.sequence.find(text=True, recursive=False).strip()
    document_filename = filing_document.filename.find(text=True, recursive=False).strip()
    document_description = filing_document.description.find(text=True, recursive=False).strip()
    
    # initalize our document dictionary
    master_document_dict[document_id] = {}
    
    # add the different parts, we parsed up above.
    master_document_dict[document_id]['document_sequence'] = document_sequence
    master_document_dict[document_id]['document_filename'] = document_filename
    master_document_dict[document_id]['document_description'] = document_description
    
    # store the document itself, this portion extracts the HTML code. We will have to reparse it later.
    master_document_dict[document_id]['document_code'] = filing_document.extract()
    

## initialize a master dict for 10-k

we'll want: 

- company name
- year
- however we're going to split up the individual 10-ks


In [4]:
#initialize master 10-k dict
tenK_document_dict = {}

# tenK_text = soup.find('text').extract()
tenK_text = master_document_dict['10-K']['document_code'].find('text').extract()

# parts_to_split = ['PART I ',
#                   'PART II\n',
#                   'PART III ',
#                  ]

parts_to_split = ['BUSINESS ',
                  'RISK FACTORS ',
                  'UNRESOLVED STAFF COMMENTS ',
                  'PROPERTIES ',
                  'LEGAL PROCEEDINGS ',
                  'RESERVED ',
                  'PART II\n'
                 ]




#get different parts of document: I-IV
text_split_into_parts = []

# for parts I-IV
for part in parts_to_split:
    next_part = tenK_text.find('b', text=part)
    print(next_part)
    text_split_into_parts.append(next_part)

    
#convert all parts to string
all_parts = [str(part) for part in text_split_into_parts]

#prep the document text for splitting - convert to string
tenK_string = str(tenK_text)

#defing the regex delimeter pattern
regex_delimiter_pattern = '|'.join(map(re.escape, all_parts))

#split doc on each break
split_tenK_string = re.split(regex_delimiter_pattern, tenK_string)

#store parts in master dict
# tenK_document_dict['Part I'] = split_tenK_string[1]
# tenK_document_dict['Part II'] = split_tenK_string[2]
# tenK_document_dict['Part III'] = split_tenK_string[3]
tenK_document_dict['Business'] = split_tenK_string[1]
tenK_document_dict['Risk Factors'] = split_tenK_string[2]
tenK_document_dict['Unresolved Staff Comments'] = split_tenK_string[3]
tenK_document_dict['Properties'] = split_tenK_string[4]
tenK_document_dict['Legal Proceedings'] = split_tenK_string[5]
tenK_document_dict['Reserved'] = split_tenK_string[6]








<b>BUSINESS </b>
<b>RISK FACTORS </b>
<b>UNRESOLVED STAFF COMMENTS </b>
<b>PROPERTIES </b>
<b>LEGAL PROCEEDINGS </b>
<b>RESERVED </b>
<b>PART II
</b>


In [36]:
t = bsoup_extract_from_string(tenK_document_dict['Business'])

In [37]:
def str_rep(string, old_list, new_list):
    for i in range(len(old_list)):
        string = string.replace(old_list[i], new_list[i])
#     string = string.replace(old, new)
    return string


def remove_extra_spaces(string):
    return " ".join(string.split())

t = str_rep(t, ['\n', '\xa0', '\x95', 
                '\x92', '\x94', '\x97', '\x93'], [' ', ' ', ' ', 
                                          '', ' ', ' ', ' '])
t = remove_extra_spaces(t)
t

'We design, develop, manufacture and sell high-performance fully electric vehicles and advanced electric vehicle powertrain components. We own our sales and service network and have operationally structured our business in a manner that we believe will enable us to rapidly develop and launch advanced electric vehicles and technologies. We believe our vehicles, electric vehicle engineering expertise, and operational structure differentiates us from incumbent automobile manufacturers. We are the first company to commercially produce a federally-compliant electric vehicle, the Tesla Roadster, which achieves a market-leading range on a single charge combined with attractive design, driving performance and zero tailpipe emissions. Our Tesla Roadster offers impressive acceleration and performance without producing any tailpipe emissions. The Tesla Roadsters proprietary electric vehicle powertrain system is the foundation of our business and, with design enhancements, will also form the basis

In [16]:
def bsoup_extract_from_string(string):
    return ' '.join(BeautifulSoup(string, "html.parser").findAll(text=True))

In [5]:
len(split_tenK_string)

8

In [6]:
type(tenK_text)

bs4.element.Tag

In [38]:
text = ' '.join(BeautifulSoup(tenK_document_dict['Properties'], "html.parser").findAll(text=True))

In [ ]:
tenK_text.findAll('b', text=True)

## Parsing out PART I info

### Part 1 is divided into:
- Business
- Risk Factors
- Unresolved Staff Comments
- Properties
- Legal Proceedings
- Reserved

We will probably split on the actual bolded words as these should be constant for all SEC 10-k filings

In [ ]:
part_one_sections = ['BUSINESS ', 
                     'RISK FACTORS ',
                     'UNRESOLVED STAFF COMMENTS ', 
                     'PROPERTIES',
                     'LEGAL PROCEEDINGS ',
                     'RESERVED '
                    ]


#get different parts of document: I-IV
partOne_split_into_parts = []

# for parts I-IV
for part in partOne_split_into_parts:
    next_part = tenK_text.find('b', text=part)
    print(next_part)
    text_split_into_parts.append(next_part)

    
#convert all parts to string
all_parts = [str(part) for part in text_split_into_parts]

#prep the document text for splitting - convert to string
tenK_string = str(tenK_text)

#defing the regex delimeter pattern
regex_delimiter_pattern = '|'.join(map(re.escape, all_parts))

#split doc on each break
split_tenK_string = re.split(regex_delimiter_pattern, tenK_string)

#store parts in master dict
tenK_document_dict['Part I'] = split_tenK_string[1]
tenK_document_dict['Part II'] = split_tenK_string[2]
tenK_document_dict['Part III'] = split_tenK_string[3]

In [ ]:
text = ' '.join(BeautifulSoup(some_html_string, "html.parser").findAll(text=True))

## Reading tables